In [2]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
batchsize=8

In [5]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory(r'D:\dataset\Prepared data\Traindata',
                                target_size=(80,80),batch_size=8,class_mode='categorical' ,subset='training')

validation_data= train_datagen.flow_from_directory(r'D:\dataset\Prepared data\Traindata',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 64675 images belonging to 2 classes.
Found 16167 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r'D:\dataset\Prepared data\testdata',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4056 images belonging to 2 classes.


In [15]:
 bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3))) 
    

87916544/87910968 [==============================] - 290s 3us/step


In [16]:
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [18]:
model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 39, 39, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 39, 39, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 39, 39, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [22]:
checkpoint = ModelCheckpoint(r'D:\dataset\model',
                            monitor='val_loss',save_best_only=True,verbose=3)
earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
8084/8084 [==============================] - ETA: 0s - loss: 0.2098 - accuracy: 0.9180
Epoch 00001: val_loss improved from inf to 0.17774, saving model to D:\dataset\model
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: D:\dataset\model\assets
8084/8084 [==============================] - 2207s 273ms/step - loss: 0.2098 - accuracy: 0.9180 - val_loss: 0.1777 - val_accuracy: 0.9251
Epoch 2/5
8083/8084 [============================>.] - ETA: 0s - loss: 0.1784 - accuracy: 0.9313
Epoch 00002: val_loss improved from 0.17774 to 0.15883, saving model to D:\dataset\model
INFO:tensorflow:Assets written to: D:\dataset\model\assets
8084/8084 [==============================] - 548s 68ms/step - loss

In [1]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

NameError: name 'model' is not defined

In [25]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

0.1859501302242279
0.9249087572097778


In [26]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

0.1382990926504135
0.9469501376152039
